In [5]:
"""
Python implementation of Neighborhood Component Feature Selection
Yang, W., Wang, K., & Zuo, W. (2012). Neighborhood Component Feature Selection
for High-Dimensional Data. Journal of Computers, 7(1).
https://doi.org/10.4304/jcp.7.1.161-168
Author : Dakota Hawkins
"""

import numpy as np
from scipy import spatial
from sklearn import base, model_selection

def pairwise_feature_distance(data_matrix, metric='cityblock'):
#def pairwise_feature_distance(data_matrix, metric='euclidean'):
    """
    Calculate the pairwise distance between each sample in each feature.
    
    Parameters
    ----------
    data_matrix : numpy.ndarray
        An (N x M) data matrix where N is the number of samples and M is the
        number of features.
    metric : str, optional
        Distance metric to use. The default is 'cityblock'.
    
    Returns
    -------
    numpy.ndarray
        An (M x N X N) numpy array where array[0] is the distance matrix
        representing pairwise distances between samples in feature 0.
    """
    # matrix to hold pairwise distances between samples in each feature
    dists = np.zeros((data_matrix.shape[1],
                        data_matrix.shape[0], data_matrix.shape[0]))
    for j in range(data_matrix.shape[1]):
        dists[j] = spatial.distance.squareform(
                        spatial.distance.pdist(data_matrix[:, j].reshape(-1, 1),
                                               metric=metric))
    return dists


class NCFSOptimizer(object):
    """Gradient ascent/descent optimization following NCFS protocol."""

    def __init__(self, alpha=0.01):
        """
        Gradient ascent/descent optimization following NCFS protocol.
        
        Parameters
        ----------
        alpha : float, optional
            Initial step size. The default is 0.01.
        Attributes
        ----------
        alpha
        Methods
        -------
        get_steps(gradients) : get gradient deltas for each feature gradient.
        """
        self.alpha = alpha

    def get_steps(self, gradients, loss=None, *args):
        """
        Calculate gradient deltas for each feature gradient.
        
        Parameters
        ----------
        gradients : numpy.ndarray
            Calculated gradient delta of objective function with respect to
            given feature.
        loss : float
            Difference between objective function at t and t - 1.
        
        Returns
        -------
        numpy.ndarray
            Gradient steps for each feature gradient.
        """
        steps = self.alpha * gradients
        if loss is not None:
            if loss > 0:
                self.alpha *= 1.01
            else:
                self.alpha *= 0.4
       
        return steps

    
class KernelMixin(object):
    """Base class for kernel functions."""

    def __init__(self, sigma, reg, weights):
        """
        Base class for kernel functions.
        Parameters
        ----------
        sigma : float
            Scaling parameter sigma as mentioned in original paper.
        reg : float
            Regularization parameter.
        weights : numpy.ndarray
            Initial vector of feature weights.
        Methods
        -------
        transform():
            Apply the kernel tranformation to a distance matrix.
        gradients():
            Get current gradients for each feature
        update_weights():
            Update feature weights to a new value.
        """
        self.sigma = sigma
        self.reg = reg
        self.weights = weights

    def transform(self, distance_matrix):
        """Apply a kernel transformation to a distance matrix."""
        return distance_matrix

    def gradients(self, p_reference, feature_distances, class_matrix):
        """
        Calculate gradients with respect to feature weights.
        Calculates the gradient vector of the objective function with respect
        to feature weights. Objective function is the same outlined in the
        original NCFS paper.
        
        Parameters
        ----------
        p_reference : numpy.ndarray
            A (sample x sample) probability matrix with p_ij represents the
            probability of selecting sample j as a reference for sample i.
        feature_distances : numpy.ndarray
            A (feature x sample x sample) tensor, holding per feature sample
            distances, such that [l, i, j] indexes the distance between
            sample i and j in feature l.
        class_matrix : numpy.ndarray
            A one-hot (sample x sample) matrix where c_ij = 1 indicates sample
            i and sample j are in the same class, and c_ij = 0 otherwise.
        
        Returns
        -------
        numpy.ndarray
            Gradient vector for each feature with respect to the objective
            function.
        """
        deltas = np.zeros(self.weights.size, dtype=np.float64)
        # calculate probability of correct classification
        # check class mat one hot shit
        p_correct = np.sum(p_reference * class_matrix, axis=1)
        # caclulate weight adjustments for each feature
        for l in range(self.weights.size):
            # weighted distance matrix D_ij = d_ij * p_ij, p_ii = 0
            d_mat = feature_distances[l] * p_reference
            # calculate p_i * sum(D_ij), j from 0 to N
            all_term = p_correct * d_mat.sum(axis=1)
            # weighted in-class distances using adjacency matrix,
            in_class_term = np.sum(d_mat * class_matrix, axis=1)
            sample_terms = all_term - in_class_term
            # calculate delta following gradient ascent 
            deltas[l] = 2 * self.weights[l] \
                      * ((1 / self.sigma) * sample_terms.sum() - self.reg)
        return deltas

    def update_weights(self, new_weights):
        """
        Update feature weights.
        
        Parameters
        ----------
        new_weights : numpy.ndarray
            New feature weights.
        """
        self.weights = new_weights


class ExponentialKernel(KernelMixin):
    """Class for the exponential kernel function used in original NCFS paper."""

    def __init__(self, sigma, reg, weights):
        """
        Class for the exponential kernel function used in original NCFS paper.
        Extends KernelMixin.
        Parameters
        ----------
        sigma : float
            Scaling parameter sigma as mentioned in original paper.
        reg : float
            Regularization parameter.
        weights : numpy.ndarray
            Initial vector of feature weights.
        Methods
        -------
        transform():
            Apply the kernel tranformation to a distance matrix.
        gradients():
            Get current gradients for each feature
        update_weights():
            Update feature weights to a new value.
        """
        super(ExponentialKernel, self).__init__(sigma, reg, weights)

    def transform(self, distance_matrix):
        """
        Apply the kernel function to each sample distance in a distance matrix.
        
        Parameters
        ----------
        distance_matrix : numpy.ndarray
            A (sample x sample) distance matrix.
        
        Returns
        -------
        numpy.ndarray
            A (sampel x sample) kernel distance matrix.
        """
        return np.exp(-1 * distance_matrix / self.sigma, dtype=np.float64)


class NCFS(base.BaseEstimator, base.TransformerMixin):
    """
    Class to perform Neighborhood Component Feature Selection 
    References
    ----------
    Yang, W., Wang, K., & Zuo, W. (2012). Neighborhood Component Feature
    Selection for High-Dimensional Data. Journal of Computers, 7(1).
    https://doi.org/10.4304/jcp.7.1.161-168
    """

    def __init__(self, alpha=0.1, sigma=1, reg=1, eta=0.001,
                 metric='cityblock', kernel='exponential', solver='ncfs'):
        """
        Class to perform Neighborhood Component Feature Selection 
        Parameters
        ----------
        alpha : float, optional
            Initial step length for gradient ascent. Should be between 0 and 1.
            Default is 0.1.
        sigma : float, optional
            Kernel width. Default is 1.
        reg : float, optional
            Regularization constant. Lambda in the original paper. Default is 1.
        eta : float, optional
            Stopping criteria for iteration. Threshold for difference between
            objective function scores after each iteration. Default is 0.001.
        metric : str, optional
            Metric to calculate distances between samples. Must be a scipy
            implemented distance and accept a parameter 'w' for a weighted
            distance. Default is manhattan/cityblock distance.
        kernel : str, optional
            Method to calculate kernel distance between samples. The only
            accepted value is 'exponential', as used in the original NCFS paper.
        solver : str, optional
            Method to perform gradient ascent. The only supported value is
            'ncfs', and follows the line-search method outlined in the original
            NCFS paper.
        Attributes:
        ----------
        alpha : float
            Step length for gradient ascent. Varies during training.
        sigma : float
            Kernel width.
        reg : float
            Regularization constant. Lambda in the original paper.
        eta : float
            Stopping criteria for iteration. Threshold for difference between
            objective function scores after each iteration.
        metric : str
            Distance metric to use.
        kernel : str
            Kernel function to use to calculate kernel distance.
        solver : str
            Method to use to perform gradient ascent.
        coef_ : numpy.array
            Feature weights. Unimportant features tend toward zero.
        score_ : float
            Objective function score at the end of fitting.
        Methods
        -------
        fit : Fit feature weights given a particular data matrix and sample
            labels.
        References
        ----------
        Yang, W., Wang, K., & Zuo, W. (2012). Neighborhood Component Feature
        Selection for High-Dimensional Data. Journal of Computers, 7(1).
        https://doi.org/10.4304/jcp.7.1.161-168
        """
        self.alpha = alpha
        self.sigma = sigma
        self.reg = reg
        self.eta = eta 
        self.metric = metric
        self.kernel = kernel
        self.solver = solver
        self.coef_ = None
        self.score_ = None

    def fit(self, X, y):
        """
        Fit feature weights using Neighborhood Component Feature Selection.
        Fit feature weights using Neighborhood Component Feature Selection.
        Weights features in `X` by their ability to distinguish classes in `y`.
        Coefficients are set to the instance variable `self.coef_`. 
        Parameters
        ----------
        X : numpy.ndarray
            An n x p data matrix where n is the number of samples, and p is the
            number of features.
        y : numpy.array
            List of pre-defined classes for each sample in `X`.
        Returns
        -------
        NCFS
            Fitted NCFS object with weights stored in the `.coef_` instance
            variable.
        """
        if not 0 < self.alpha < 1:
            raise ValueError("Alpha value should be between 0 and 1.")
        if not isinstance(X, np.ndarray):
            raise ValueError('`X` must be two-dimensional numpy array. Got ' + 
                             '{}.'.format(type(X)))
        if len(X.shape) != 2:
            raise ValueError('`X` must be two-dimensional numpy array. Got ' + 
                             '{} dimensional.'.format(len(X.shape)))
        if not isinstance(y, np.ndarray):
            raise ValueError('`y` must be a numpy array. ' + 
                             'Got {}.'.format(type(y)))
        if y.shape[0] != X.shape[0]:
            raise ValueError('`X` and `y` must have the same row numbers.')
        n_samples, n_features = X.shape
        # initialize all weights as 1
        self.coef_ = np.ones(n_features, dtype=np.float64)

        # intialize kernel function, and assign expected feature distances 
        feature_distances = None
        if self.kernel == 'exponential':
            self.kernel_ = ExponentialKernel(self.sigma, self.reg, self.coef_)
            feature_distances = pairwise_feature_distance(X, metric=self.metric)
        else:
            raise ValueError('Unsupported kernel ' +
                             'function: {}'.format(self.kernel))

        if self.solver == 'ncfs':
            self.solver_ = NCFSOptimizer(alpha=self.alpha)
        else:
            raise ValueError('Unsupported gradient ascent method ' +
                             '{}'.format(self.solver))
        # construct adjacency matrix of class membership for matrix mult. 
        class_matrix = np.zeros((n_samples, n_samples), np.float64)
        for i in range(n_samples):
            for j in range(n_samples):
                if y[i] == y[j] and i != j:
                    class_matrix[i, j] = 1

        objective, loss = 0, np.inf
        while abs(loss) > self.eta:
            new_objective = self.__fit(X, class_matrix, objective,
                                       feature_distances)
            loss = objective - new_objective
            objective = new_objective

        self.score_ = objective
        return self

    def transform(self, X):
        """
        Transform features according to their learned weights.
        
        Parameters
        ----------
        X : numpy.ndarray
            An `(n x p)` data matrix where `n` is the number of samples, and `p`
            is the number of features. Features number and order should be the
            same as those used to fit the model.  
        
        Raises
        ------
        RuntimeError
            Raised if the NCFS object has not been fit yet.
        ValueError
            Raided if the number of feature dimensions does not match the
            number of learned weights.
        
        Returns
        -------
        numpy.ndarray
            Transformed data matrix calculated by multiplying each feature by
            its learnt weight.
        """

        if self.coef_ is None:
            raise RuntimeError('NCFS is not fit. Please fit the ' +
                               'estimator by calling `.fit()`.')
        if X.shape[1] != len(self.coef_):
            raise ValueError('Expected data matrix `X` to contain the same' + 
                             'number of features as learnt feature weights.')
        return X*self.coef_
                          
    def __fit(self, X, class_matrix, objective, feature_distances):
        """
        Underlying method to fit NCFS model.
        
        Parameters
        ----------
        X : numpy.ndarray
            A sample by feature data matrix.
        class_matrix : numpy.ndarray
            A sample by sample one-hot matrix marking samples in the same class.
        objective : float
            Objective score reached from past iteration
        
        Returns
        -------
        float
            Objective reached by current iteration.
        """
        # calculate D_w(x_i, x_j): w^2 * |x_i - x_j] for all i,j
        distances = spatial.distance.pdist(X, metric=self.metric,
                                           w=np.power(self.coef_, 2))
        

        # organize as distance matrix
        distances = spatial.distance.squareform(distances)
        
        # calculate K(D_w(x_i, x_j)) for all i, j pairs
        p_reference = self.kernel_.transform(distances)
        # set p_ii = 0, can't select self in leave-one-out
        for i in range(p_reference.shape[0]):
            p_reference[i, i] = 0
        
        # add pseudocount if necessary to avoid dividing by zero
        row_sums = p_reference.sum(axis=1)
        
        n_zeros = sum(row_sums == 0)
        
        if n_zeros > 0:
            print('Adding pseudocounts to distance matrix to avoid ' +
                    'dividing by zero.')
            if n_zeros == len(row_sums):
                pseudocount = np.exp(-20)
            else:
                pseudocount = np.max(row_sums,np.finfo(float).tiny)
                #pseudocount = np.min(row_sums)
            
            row_sums += pseudocount
        
        scale_factors = 1 / (row_sums)
        
        #scale_factors = np.where(scale_factors==np.inf,1.00000000e+308,scale_factors)
        #print('scale_factors after',scale_factors)
        p_reference = (p_reference.T * scale_factors).T

        # caclulate weight adjustments
        gradients = self.kernel_.gradients(p_reference, feature_distances,
                                                 class_matrix)
            
        # calculate objective function
        new_objective = (np.sum(p_reference * class_matrix) \
                        - self.reg * np.dot(self.coef_, self.coef_))
        # calculate loss from previous objective function
        loss = new_objective - objective
        # update weights
        deltas = self.solver_.get_steps(gradients, loss)
        self.coef_ = self.coef_ + deltas
        self.kernel_.update_weights(self.coef_)
        # return objective score for new iteration
        return new_objective


def toy_dataset(n_features=1000):
    """
    Generate a toy dataset with features from the original NCFS paper.
    
    Generate a toy dataset with features from the original NCFS paper. Signal
    features are in the first index, and the 10th percent index (e.g.
    :math:`0.1 * N`). See original paper for specific parameter values for
    signal/noise features.
    
    Parameters
    ----------
    n_features : int, optional
        Number of total features. Two of these features will feature signal,
        the other N - 2 will be noise. The default is 1000.
    
    Returns
    -------
    tuple (X, y)
        X : numpy.array
            Simulated dataset with 200 samples (rows) and N features. Features
            are scaled between 0 and 1.
        y : numpy.array
            Class membership for each sample in X.
    """

    class_1 = np.zeros((100, 2))
    class_2 = np.zeros((100, 2))
    cov = np.identity(2)
    for i in range(100):
        r1, r2 = np.random.rand(2)
        if r1 > 0.5:
            class_1[i, :] = np.random.multivariate_normal([-0.75, -3], cov)
        else:
            class_1[i, :] = np.random.multivariate_normal([0.75, 3], cov)
        if r2 > 0.5:
            class_2[i, :] = np.random.multivariate_normal([3, -3], cov)
        else:
            class_2[i, :] = np.random.multivariate_normal([-3, 3], cov)
    class_data = np.vstack((class_1, class_2))
    n_irrelevant = n_features - 2
    second_idx = int(0.1*(n_features)) - 1
    bad_features = np.random.normal(loc=0, scale=np.sqrt(20),
                                    size=(200, n_irrelevant))
    data = np.hstack((class_data[:, 0].reshape(-1, 1),
                      bad_features[:, :second_idx],
                      class_data[:, 1].reshape(-1, 1),
                      bad_features[:, second_idx:]))
    classes = np.array([0]*100 + [1]*100)
    # scale between 0 - 1
    x_std = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
    return x_std, classes

def main():
    X, y = toy_dataset()
    f_select = NCFS(alpha=0.001, sigma=1, reg=1, eta=0.001,
                    metric='cityblock', kernel='exponential', solver='ncfs')
    from timeit import default_timer as timer
    start = timer()
    f_select.fit(X, y)
    end = timer()
    print("Execution time in seconds: {}".format(end - start))
    print(np.argsort(-f_select.coef_)[:10])
    print(f_select.coef_[0], f_select.coef_[100])
    print("Score: {}".format(f_select.score_))

if __name__ == '__main__':
    main()

[[  0.         200.53073323 208.25412909 ... 204.32871628 210.6586709
  207.94312183]
 [200.53073323   0.         201.93910391 ... 201.97932577 202.71633878
  203.59343405]
 [208.25412909 201.93910391   0.         ... 199.96532944 207.28296604
  214.28214378]
 ...
 [204.32871628 201.97932577 199.96532944 ...   0.         207.2931612
  211.02945219]
 [210.6586709  202.71633878 207.28296604 ... 207.2931612    0.
  212.93852912]
 [207.94312183 203.59343405 214.28214378 ... 211.02945219 212.93852912
    0.        ]]
[[  0.         199.7162189  207.40233331 ... 203.51842712 209.81834871
  207.10393003]
 [199.7162189    0.         201.12434402 ... 201.16971345 201.89885013
  202.76658754]
 [207.40233331 201.12434402   0.         ... 199.16106471 206.44817608
  213.41152471]
 ...
 [203.51842712 201.16971345 199.16106471 ...   0.         206.46835761
  210.19664427]
 [209.81834871 201.89885013 206.44817608 ... 206.46835761   0.
  212.08726137]
 [207.10393003 202.76658754 213.41152471 ... 210.1

[[  0.         194.56024791 202.00846982 ... 198.39180303 204.49900259
  201.79174138]
 [194.56024791   0.         195.96628774 ... 196.0476195  196.72645752
  197.5328242 ]
 [202.00846982 195.96628774   0.         ... 194.07413793 201.16490871
  207.89891828]
 ...
 [198.39180303 196.0476195  194.07413793 ...   0.         201.24984151
  204.92644439]
 [204.49900259 196.72645752 201.16490871 ... 201.24984151   0.
  206.69590236]
 [201.79174138 197.5328242  207.89891828 ... 204.92644439 206.69590236
    0.        ]]
[[  0.         194.19402061 201.62524864 ... 198.02781143 204.1211691
  201.4144247 ]
 [194.19402061   0.         195.59989339 ... 195.68397313 196.35921942
  197.1611084 ]
 [201.62524864 195.59989339   0.         ... 193.71306755 200.78971685
  207.5072719 ]
 ...
 [198.02781143 195.68397313 193.71306755 ...   0.         200.87933651
  204.55219985]
 [204.1211691  196.35921942 200.78971685 ... 200.87933651   0.
  206.31278149]
 [201.4144247  197.1611084  207.5072719  ... 204.

[[  0.         188.34465259 195.50319984 ... 192.21662522 198.08642563
  195.38829809]
 [188.34465259   0.         189.74787964 ... 189.878957   190.49677094
  191.22538937]
 [195.50319984 189.74787964   0.         ... 187.9503888  194.79888646
  201.2504457 ]
 ...
 [192.21662522 189.878957   187.9503888  ...   0.         194.96481916
  198.57644585]
 [198.08642563 190.49677094 194.79888646 ... 194.96481916   0.
  200.1903623 ]
 [195.38829809 191.22538937 201.2504457  ... 198.57644585 200.1903623
    0.        ]]
[[  0.         187.93032663 195.06949075 ... 191.80516589 197.65897198
  194.96149231]
 [187.93032663   0.         189.33338264 ... 189.46799138 190.08174791
  190.80507201]
 [195.06949075 189.33338264   0.         ... 187.5424968  194.37467481
  200.80716059]
 ...
 [191.80516589 189.46799138 187.5424968  ...   0.         194.54611166
  198.15328052]
 [197.65897198 190.08174791 194.37467481 ... 194.54611166   0.
  199.75646539]
 [194.96149231 190.80507201 200.80716059 ... 198.

[[  0.         181.33404208 188.16416657 ... 185.25673979 190.85390412
  188.16795052]
 [181.33404208   0.         182.73516225 ... 182.92877771 183.47881457
  184.11688748]
 [188.16416657 182.73516225   0.         ... 181.05316995 187.62371817
  193.74839995]
 ...
 [185.25673979 182.92877771 181.05316995 ...   0.         187.88434928
  191.41800282]
 [190.85390412 183.47881457 187.62371817 ... 187.88434928   0.
  192.8446934 ]
 [188.16795052 184.11688748 193.74839995 ... 191.41800282 192.8446934
    0.        ]]
[[  0.         180.86835326 187.67666212 ... 184.79455396 190.37349969
  187.68847464]
 [180.86835326   0.         182.26941299 ... 182.46735229 183.01298693
  183.6450166 ]
 [187.67666212 182.26941299   0.         ... 180.59531972 187.14731256
  193.2499749 ]
 ...
 [184.79455396 182.46735229 180.59531972 ...   0.         187.41434292
  190.94261266]
 [190.37349969 183.01298693 187.14731256 ... 187.41434292   0.
  192.35646871]
 [187.68847464 183.6450166  193.2499749  ... 190.

[[  0.         173.48242916 179.94613143 ... 177.46514868 182.75497787
  180.08787045]
 [173.48242916   0.         174.8846295  ... 175.15235445 175.63111472
  176.16853406]
 [179.94613143 174.8846295    0.         ... 173.33762331 179.59530415
  185.34398131]
 ...
 [177.46514868 175.15235445 173.33762331 ...   0.         179.96524947
  183.40454251]
 [182.75497787 175.63111472 179.59530415 ... 179.96524947   0.
  184.60921483]
 [180.08787045 176.16853406 185.34398131 ... 183.40454251 184.60921483
    0.        ]]
[[  0.         172.9630061  179.40263924 ... 176.94972079 182.21927111
  179.55370174]
 [172.9630061    0.         174.36547278 ... 174.63811792 175.11243305
  175.64335887]
 [179.40263924 174.36547278   0.         ... 172.82745416 179.06449218
  184.78795512]
 ...
 [176.94972079 174.63811792 172.82745416 ...   0.         179.44175658
  182.87454375]
 [182.21927111 175.11243305 179.06449218 ... 179.44175658   0.
  184.06414011]
 [179.55370174 175.64335887 184.78795512 ... 182

...

[[ 0.          4.18727186  8.9333506  ... 11.70195545  8.66521458
  10.62086742]
 [ 4.18727186  0.         11.1295086  ... 13.90648492  5.24195616
   7.20981452]
 [ 8.9333506  11.1295086   0.         ... 11.79224384 10.31776439
   9.01784199]
 ...
 [11.70195545 13.90648492 11.79224384 ...  0.         18.04308079
  20.07080358]
 [ 8.66521458  5.24195616 10.31776439 ... 18.04308079  0.
   2.81815602]
 [10.62086742  7.20981452  9.01784199 ... 20.07080358  2.81815602
   0.        ]]
[[ 0.          4.14689548  8.89271052 ... 11.6610159   8.62202284
  10.57959877]
 [ 4.14689548  0.         11.08971069 ... 13.86630564  5.19931128
   7.16839748]
 [ 8.89271052 11.08971069  0.         ... 11.75291839 10.27655477
   8.97508813]
 ...
 [11.6610159  13.86630564 11.75291839 ...  0.         18.00262274
  20.02906476]
 [ 8.62202284  5.19931128 10.27655477 ... 18.00262274  0.
   2.7742316 ]
 [10.57959877  7.16839748  8.97508813 ... 20.02906476  2.7742316
   0.        ]]
[[ 0.          4.10838918  8.8539

[[ 0.          3.71275106  8.45623771 ... 11.22173582  8.15693549
  10.1358801 ]
 [ 3.71275106  0.         10.6624138  ... 13.4345828   4.73782968
   6.72097782]
 [ 8.45623771 10.6624138   0.         ... 11.3307496   9.833152
   8.51407658]
 ...
 [11.22173582 13.4345828  11.3307496  ...  0.         17.56847637
  19.5788487 ]
 [ 8.15693549  4.73782968  9.833152   ... 17.56847637  0.
   2.30030615]
 [10.1358801   6.72097782  8.51407658 ... 19.5788487   2.30030615
   0.        ]]
[[ 0.          3.69597736  8.43941167 ... 11.20483756  8.13894572
  10.11874177]
 [ 3.69597736  0.         10.64595189 ... 13.41793685  4.71982663
   6.70355372]
 [ 8.43941167 10.64595189  0.         ... 11.31449626  9.81600438
   8.49618507]
 ...
 [11.20483756 13.41793685 11.31449626 ...  0.         17.55176454
  19.56137272]
 [ 8.13894572  4.71982663  9.81600438 ... 17.55176454  0.
   2.28191442]
 [10.11874177  6.70355372  8.49618507 ... 19.56137272  2.28191442
   0.        ]]
[[ 0.          3.68012576  8.42351

[[ 0.          3.52735675  8.27087071 ... 11.03614859  7.95833478
   9.94662292]
 [ 3.52735675  0.         10.48119621 ... 13.25126729  4.53704157
   6.52661303]
 [ 8.27087071 10.48119621  0.         ... 11.152137    9.64349823
   8.31541816]
 ...
 [11.03614859 13.25126729 11.152137   ...  0.         17.38458753
  19.38487256]
 [ 7.95833478  4.53704157  9.64349823 ... 17.38458753  0.
   2.09660263]
 [ 9.94662292  6.52661303  8.31541816 ... 19.38487256  2.09660263
   0.        ]]
[[ 0.          3.52139453  8.2649559  ... 11.03026775  7.95198037
   9.94055186]
 [ 3.52139453  0.         10.47542252 ... 13.24542308  4.53047391
   6.52023976]
 [ 8.2649559  10.47542252  0.         ... 11.14647339  9.63739359
   8.30897092]
 ...
 [11.03026775 13.24542308 11.14647339 ...  0.         17.37872936
  19.37858524]
 [ 7.95198037  4.53047391  9.63739359 ... 17.37872936  0.
   2.09004439]
 [ 9.94055186  6.52023976  8.30897092 ... 19.37858524  2.09004439
   0.        ]]
[[ 0.          3.51581026  8.259

[[ 0.          3.46496842  8.20963487 ... 10.97573733  7.89254215
   9.88336372]
 [ 3.46496842  0.         10.42149574 ... 13.19078387  4.46728943
   6.45849418]
 [ 8.20963487 10.42149574  0.         ... 11.09401024  9.57963343
   8.24735721]
 ...
 [10.97573733 13.19078387 11.09401024 ...  0.         17.32390295
  19.31865038]
 [ 7.89254215  4.46728943  9.57963343 ... 17.32390295  0.
   2.02834885]
 [ 9.88336372  6.45849418  8.24735721 ... 19.31865038  2.02834885
   0.        ]]
[[ 0.          3.46311333  8.20786198 ... 10.97401913  7.89064227
   9.88150371]
 [ 3.46311333  0.         10.41977117 ... 13.18903163  4.46515378
   6.45637291]
 [ 8.20786198 10.41977117  0.         ... 11.09236428  9.57773808
   8.24529607]
 ...
 [10.97401913 13.18903163 11.09236428 ...  0.         17.32213826
  19.31665646]
 [ 7.89064227  4.46515378  9.57773808 ... 17.32213826  0.
   2.02635852]
 [ 9.88150371  6.45637291  8.24529607 ... 19.31665646  2.02635852
   0.        ]]
[[ 0.          3.46138574  8.206

[[ 0.          3.44600677  8.1920936  ... 10.95903834  7.87388066
   9.86462845]
 [ 3.44600677  0.         10.40445143 ... 13.17337004  4.44492887
   6.43580203]
 [ 8.1920936  10.40445143  0.         ... 11.07816598  9.56035118
   8.22594265]
 ...
 [10.95903834 13.17337004 11.07816598 ...  0.         17.30625278
  19.29807565]
 [ 7.87388066  4.44492887  9.56035118 ... 17.30625278  0.
   2.0086715 ]
 [ 9.86462845  6.43580203  8.22594265 ... 19.29807565  2.0086715
   0.        ]]
[[ 0.          3.44544573  8.19161311 ... 10.95859911  7.87338035
   9.86409302]
 [ 3.44544573  0.         10.40398504 ... 13.17288567  4.44423715
   6.43506756]
 [ 8.19161311 10.40398504  0.         ... 11.07776157  9.55978794
   8.22528871]
 ...
 [10.95859911 13.17288567 11.07776157 ...  0.         17.30575367
  19.29745649]
 [ 7.87338035  4.44423715  9.55978794 ... 17.30575367  0.
   2.00813824]
 [ 9.86409302  6.43506756  8.22528871 ... 19.29745649  2.00813824
   0.        ]]
[[ 0.          3.44492104  8.1911